In [1]:
import os
# to use a specific GPU
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [2]:
from mgtbench import AutoDetector, AutoExperiment
from mgtbench.loading.dataloader import load

#### Model-based Detector

In [3]:
# local path to the model, or model name on huggingface
model_name_or_path = '/data1/models/distilbert-base-uncased'
metric = AutoDetector.from_detector_name('LM-D', 
                                            model_name_or_path=model_name_or_path)
experiment = AutoExperiment.from_experiment_name('supervised',detector=[metric])

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /data1/models/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
'''
supported LLMs and detect categories:

categories = ['Physics', 'Medicine', 'Biology', 'Electrical_engineering', 'Computer_science', 'Literature', 'History', 'Education', 'Art', 'Law', 'Management', 'Philosophy', 'Economy', 'Math', 'Statistics', 'Chemistry']

llms = ['Moonshot', 'gpt35', 'Mixtral', 'Llama3']
'''
data_name = 'AITextDetect'
detectLLM = 'Llama3'
category = 'Art'

In [5]:
data = load(data_name, detectLLM, category)
# for demo
size = 200
data['train']['text'] = data['train']['text'][:size]
data['train']['label'] = data['train']['label'][:size]
data['test']['text'] = data['test']['text'][:size]
data['test']['label'] = data['test']['label'][:size]

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
experiment.load_data(data)
config = {'need_finetune': True,
          'need_save': False,
          'epochs': 1, # for model-based detectors
          }
res = experiment.launch(**config)
print('==========')
print('train:', res[0].train)
print('test:', res[0].test)

Calculate result for each data point
Running prediction of detector LM-D


/home/zhiyuan/miniconda3/envs/mgtbench2/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/zhiyuan/miniconda3/envs/mgtbench2/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss


Fine-tune finished
Predict training data


100%|██████████| 200/200 [00:01<00:00, 185.12it/s]


Predict testing data


100%|██████████| 200/200 [00:01<00:00, 187.62it/s]

Run classification for results
train: Metric(acc=0.68, precision=0.6374269005847953, recall=0.9819819819819819, f1=0.7730496453900709, auc=0.9412389918007896)
test: Metric(acc=0.615, precision=0.5535714285714286, recall=0.9789473684210527, f1=0.7072243346007605, auc=0.9258145363408521)


#### Metric-based Detector

In [7]:
# log-likelihood detector
model_name_or_path = '/data1/zzy/gpt2-medium'
metric = AutoDetector.from_detector_name('ll', 
                                            model_name_or_path=model_name_or_path)
experiment = AutoExperiment.from_experiment_name('threshold',detector=[metric])

In [8]:
data_name = 'AITextDetect'
detectLLM = 'gpt35'
category = 'Art'

In [9]:
data = load(data_name, detectLLM, category)
# for demo
size = 200
data['train']['text'] = data['train']['text'][:size]
data['train']['label'] = data['train']['label'][:size]
data['test']['text'] = data['test']['text'][:size]
data['test']['label'] = data['test']['label'][:size]

data['train']['text'][0], data['train']['label'][0]

Repo card metadata block was not found. Setting CardData to empty.


('Calligraphy and Silk Painting in Vietnamese Art\n\nDespite the decline in use among scholars and elites, calligraphy remains an important part of Vietnamese culture. It is commonly featured in calligraphy hangings, which are made by village teachers or scholars and often display poetry, folk sayings, or single words. Even those who cannot read or write often commission scholars to write prayers that are then burned at temple shrines.\n\nAnother popular form of art in Vietnam is silk painting. This medium is known for the mystical atmosphere it can convey. In the 19th and 20th centuries, Vietnamese art absorbed French influence, resulting in a more liberal and modern use of the medium.\n\nOverall, calligraphy and silk painting play significant roles in Vietnamese art. While calligraphy serves as a way to display both written language and artistry, silk painting creates a unique, ethereal aesthetic that is beloved by many.',
 1)

In [10]:
experiment.load_data(data)
res = experiment.launch()
print('==========')
print('train:', res[0].train)
print('test:', res[0].test)

Calculate result for each data point
Running prediction of detector ll
Predict training data


100%|██████████| 200/200 [00:02<00:00, 89.61it/s]


Predict testing data


100%|██████████| 200/200 [00:02<00:00, 89.56it/s]

Run classification for results
train: Metric(acc=0.75, precision=0.7666666666666667, recall=0.7040816326530612, f1=0.7340425531914894, auc=0.8145258103241296)
test: Metric(acc=0.695, precision=0.6767676767676768, recall=0.6979166666666666, f1=0.6871794871794872, auc=0.7933693910256411)


#### Fast-DetectGPT


In [3]:
scoring_model_name_or_path = '/data1/zzy/gpt2-medium'
reference_model_name_or_path = '/data1/zzy/gpt2-medium'

fastDetectGPT = AutoDetector.from_detector_name('fast-detectGPT', 
                                            scoring_model_name_or_path=scoring_model_name_or_path,
                                            reference_model_name_or_path= reference_model_name_or_path)
experiment = AutoExperiment.from_experiment_name('perturb', detector=[fastDetectGPT])

In [6]:
data_name = 'AITextDetect'
detectLLM = 'Llama3'
category = 'Biology'
data = load(data_name, detectLLM, category)
# for demo
size = 200
data['train']['text'] = data['train']['text'][:size]
data['train']['label'] = data['train']['label'][:size]
data['test']['text'] = data['test']['text'][:size]
data['test']['label'] = data['test']['label'][:size]

loading human data
loading machine data
data loaded


parsing data: 100%|██████████| 6962/6962 [00:00<00:00, 18134.88it/s]


In [7]:
experiment.load_data(data)
res = experiment.launch()
print('==========')
print('train:', res[0].train)
print('test:', res[0].test)

Calculate result for each data point
Running prediction of detector fast-detectGPT
Predict training data


Detecting: 100%|██████████| 200/200 [00:02<00:00, 68.29it/s]


Predict testing data


Detecting: 100%|██████████| 200/200 [00:02<00:00, 69.23it/s]

Run classification for results
train: Metric(acc=0.78, precision=0.7956989247311828, recall=0.7474747474747475, f1=0.7708333333333334, auc=0.8527352735273528, conf_m=None)
test: Metric(acc=0.815, precision=0.8764044943820225, recall=0.75, f1=0.8082901554404145, auc=0.9102564102564102, conf_m=None)
